In [ ]:
# 1) (Optional) Mount Google Drive to persist checkpoints
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2) Clone the repository (or pull if already present)
import os
if not os.path.exists('270FT'):
    !git clone https://github.com/oleeveeuh/270FT.git
%cd 270FT

In [ ]:
# 3) Install required Python packages (may take a few minutes).
# bitsandbytes requires a CUDA-compatible runtime; if installation fails, try a different Colab runtime or match a wheel to the CUDA version.
!pip install -q transformers datasets peft evaluate pyyaml huggingface_hub wandb bitsandbytes

If bitsandbytes or other installs fail due to CUDA mismatches, try switching the Colab runtime GPU type (or use a notebook with a supported CUDA version). You can also skip installing `bitsandbytes` if you only want CPU runs, but the full QLoRA workflow requires a CUDA GPU.

In [ ]:
# 4) (Optional) Preprocess raw data into processed JSONL files.
# This will create/update data/processed/train.jsonl, validation.jsonl, and test.jsonl
!python preprocess/load_and_prepare.py --raw_dir data/raw --processed_dir data/processed --validation_split 0.15 --test_split 0.15

In [ ]:
# 5) (Optional) Inspect processed files sizes and line counts
!ls -lh data/processed || true
!wc -l data/processed/*.jsonl || true

In [ ]:
# 6) Provide Hugging Face token (if required).
# Use this cell to securely input a token if you need to access gated models.
from getpass import getpass
token = getpass('Hugging Face token (leave blank if not needed): ')
import os
if token:
    os.environ['HF_TOKEN'] = token
    print('HF_TOKEN set in environment')
else:
    print('No token provided; proceeding without HF token')

In [ ]:
# 6b) Disable W&B interactive logging to avoid login prompts during automated runs
import os
os.environ['WANDB_MODE'] = 'offline'
print('W&B offline mode set')

Now run the full training script below. The script `training/train_dual_lora.py` will look for `data/processed/train.jsonl`, `validation.jsonl`, and `test.jsonl` in `data/processed` and will use the models listed in `configs/training_config.yaml`. Ensure those files are present (see the preprocessing step above).

In [ ]:
# 7) Run the training script (this will start QLoRA fine-tuning).
# Note: training will log to W&B if enabled in the config; consider disabling W&B or logging in first.
!python training/train_dual_lora.py

Troubleshooting tips:
- If you see `FileNotFoundError` complaining about missing `test.jsonl`, re-run the preprocessing cell.
- If you get `bitsandbytes` import errors, try installing a different `bitsandbytes` wheel that matches the Colab CUDA runtime or switch runtime.
- If training runs out of VRAM, reduce `batch_size` in `configs/training_config.yaml` or switch to a larger GPU.

Next steps:
- If you want, I can generate a copy of this notebook as an `.ipynb` file in the repo (already created) and also add `run_colab.sh` or config presets optimized for T4/A100.